In [ ]:
'''
import torch
import torchvision.transforms as T
from torchvision.models.segmentation import deeplabv3_resnet101
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

# 设备选择（如果有 GPU）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 加载预训练的 DeepLabV3+ 模型
model = deeplabv3_resnet101(pretrained=True).to(device)
model.eval()
'''


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DeepLabV3_ResNet101_Weights.COCO_WITH_VOC_LABELS_V1`. You can also use `weights=DeepLabV3_ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/deeplabv3_resnet101_coco-586e9e4e.pth" to /root/.cache/torch/hub/checkpoints/deeplabv3_resnet101_coco-586e9e4e.pth
100%|██████████| 233M/233M [00:02<00:00, 85.7MB/s]


DeepLabV3(
  (backbone): IntermediateLayerGetter(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Se

In [ ]:
# 安装必要的依赖
!pip install ultralytics opencv-python-headless

In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO

# 定义输入图片路径和关键词列表
input_image_path = "image.jpg"
keywords = ["dog", "person"]

# 加载 YOLOv8 分割模型（如果没有会自动下载）
# 注意：这里使用的是 yolov8n-seg.pt，若需要更高精度可更换其他权重
model = YOLO("yolov8n-seg.pt")

# 对输入图片进行检测（分割）
results = model(input_image_path)

# 读取原图
img = cv2.imread(input_image_path)
height, width = img.shape[:2]

# 初始化字典，存储每个关键词的合并 mask（全零矩阵）
combined_masks = {k: np.zeros((height, width), dtype=np.uint8) for k in keywords}

# 遍历检测结果，提取分割 mask
for result in results:
    if result.masks is not None:
        # 遍历每个检测结果
        for i in range(len(result.boxes)):
            # 获取类别索引和对应标签
            class_id = int(result.boxes.cls[i])
            label = model.names[class_id]
            if label in keywords:
                # 获取当前检测对应的 mask（结果为 tensor）
                mask = result.masks.data[i].cpu().numpy()
                # 二值化 mask（阈值可调整）
                mask = (mask > 0.5).astype(np.uint8) * 255
                # 若 mask 尺寸与原图不同，则调整大小
                if mask.shape[0] != height or mask.shape[1] != width:
                    mask = cv2.resize(mask, (width, height), interpolation=cv2.INTER_NEAREST)
                # 合并同一类别的 mask（按位或操作）
                combined_masks[label] = cv2.bitwise_or(combined_masks[label], mask)

# 保存每个关键词对应的 mask 图片
for label, mask in combined_masks.items():
    mask_path = f"mask_{label}.png"
    cv2.imwrite(mask_path, mask)
    print(f"Saved mask for '{label}' as {mask_path}")



image 1/1 /content/image.jpg: 448x640 1 person, 1 truck, 1 dog, 213.5ms
Speed: 5.4ms preprocess, 213.5ms inference, 14.4ms postprocess per image at shape (1, 3, 448, 640)
Saved mask for 'dog' as mask_dog.png
Saved mask for 'person' as mask_person.png
